# CSV files 1: reading a CSV file
By the end of this lecture you will be able to:
- set the column names when reading a CSV file
- specify how to parse a CSV file
- specify a dtype schema  when reading a CSV file
- modify CPU and memory usage when reading a CSV file

Warning: this is a long lecture as we go through the full CSV parsing process!

## What is a CSV file?
A CSV file is:
- a text file that uses a comma (or other delimiter) to separate values
- a file where data is ordered in rows rather than columns
- a file where the only potential metadata is a header row of column names - no type information for each column is specified

In [ ]:
import polars as pl

In [ ]:
csv_file = "../data/titanic.csv"

We read this CSV file as we have read it many times before!

In [ ]:
df = pl.read_csv(csv_file)
df.head(3)

## CSV parsers

CSV files are text and so must be parsed to:
1. get the column names
2. split each row into columns
3. infer the dtype of each column

## Header and column names
By default Polars takes the first row of a CSV as the header to set the column names.

### No header
If the first row is not a header we can set `has_header = False` and the column names are `column_1` etc

In [ ]:
(
    pl.read_csv(
        csv_file, 
        has_header=False
    )
    .head(2)
)

### Rename columns
We can rename columns immediately after the CSV is parsed with `new_columns`.

In this example we rename the first column in lowercase

In [ ]:
(
    pl.read_csv(
        csv_file,
        new_columns=['passengerid']
    )
    .head(2)
)

### Skip rows after the header
We skip rows after the header is parsed with `skip_rows_after_header`. In this example it drops the first row of data

In [ ]:
(
    pl.read_csv(
        csv_file,
        skip_rows_after_header=1
    )
    .head(2)
)

### Skip rows to the header

If the header is on line `N` of the CSV we can set `skip_rows = N - 1`.

In this example we set line 2 of the CSV as the header

In [ ]:
(
    pl.read_csv(
        csv_file,
        skip_rows=1
    )
    .head(2)
)

If header names are duplicated (as with columns 7 and 8 of this example) Polars adds `_duplicated_0` to the column name

## Parsing CSVs
In the following examples we simulate CSV files with Python strings like this

In [ ]:
CSV_string = b"A,B,C\n0,1,2\n"

- The newline `\n` character shows the line breaks in the simulated CSV file
- The `b` before the start of the string converts the string to bytes so that it can be passed to `pl.read_csv` (otherwise if we pass a `str` Polars thinks it is a file path to a CSV and raises an `Exception`)

In [ ]:
pl.read_csv(CSV_string)

### Delimiter
Polars assumes the delimiter is a `,`, but this can be changed with the `sep` argument.

In this example we have a CSV with tab (`\t`) separated data rather than comma-separated data

In [ ]:
tab_CSV_string = b"A\tB\tC\n0\t1\t2\n"

(
    pl.read_csv(
        tab_CSV_string,
        separator="\t"
    )
)

### Comment lines

Comment lines that start with a certain character in the CSV are ignored by setting the `comment_prefix`. Here we have a comment row after the header

In [ ]:
comment_CSV_string = b"a,b,c\n#Comment\n0,1,2\n"
(
    pl.read_csv(
        comment_CSV_string,
        comment_prefix="#"
    )
)

### Quotes
Quotes in the CSV are indicated with the `quote_char`. 

In this example we have quotes because a text entry contains a comma

In [ ]:
quote_CSV_string = b'name,age\n"Armstrong,Neil",39\n'
(
    pl.read_csv(
        quote_CSV_string,
        quote_char='"'
    )
)

### Choosing columns
We can restrict which columns are in the `DataFrame` with the `columns` argument

In [ ]:
CSV_string = b"A,B,C\n0,1,2\n"
(
    pl.read_csv(
        CSV_string,
        columns=["A","C"]
    )
)

We can also do this in lazy mode with the projection pushdown optimisation

In [ ]:
(
    pl.scan_csv(CSV_string)
    .select("A","C")
    .head(2)
    .collect()
)

## Inferring the dtypes
Unlike the Parquet files we meet later, CSVs do not store the dtype of each column. 

Therefore Polars infers the dtype of each column in the CSV. To do this Polars:
- reads the first 100 lines
- if a dtype can be inferred it sets the dtype for that column
- if a consistent dtype cannot be inferred then a `ComputeError` exception is raised

### Number of rows to infer the dtypes
We can adjust the number of lines used for type inference.

In the Titanic CSV the `Age` column starts off with 57 integers before there is a decimal value in line 58.

If we try to set `infer_schema_length` lower than 58  Polars raises a `ComputeError` because it infers an integer dtype and then encounters a float on line 58 (you can check this by reducing the `infer_schema_length` value here)

In [ ]:
(
    pl.read_csv(
        csv_file,
        infer_schema_length=58
    )
    .head(2)
)

If you are getting errors from parsing the schema it is worth increasing `infer_schema_length` to see if that resolves the error.

### Setting the schema
We can also define the full schema explicitly with a `dict` when reading the CSV. With the `schema` argument set Polars does not do the schema inferrence step described above.

In this example we read in integers and floats as 32-bit dtypes

In [ ]:
(
    pl.read_csv(
        csv_file,
        schema={
            "PassengerId": pl.Int32,
            "Survived": pl.Int32,
            "Pclass": pl.Int32,
            "Name": pl.String,
            "Sex": pl.String,
            "Age": pl.Float32,
            "SibSp": pl.Int32,
            "Parch": pl.Int32,
            "Ticket": pl.String,
            "Fare": pl.Float32,
            "Cabin": pl.String,
            "Embarked": pl.String,
        },
    )
    .head(2)
)

However, with `schema` we need to pass the schema for all columns, which can be tedious.

We can instead use `schema_overrides` to override the inferred schema for specific columns

In [ ]:
(
    pl.read_csv(
        csv_file,
        schema_overrides={
            "PassengerId": pl.Int32,
            "Survived": pl.Int32,
            "Pclass": pl.Int32,
        },
    )
    .head(2)
)

## Handling mixed types and exceptions
In this example we have a CSV file that will raise an exception as the values in the first column are:
- `1.0` which looks like a float and 
- `a` which is a string

In [ ]:
mixed_CSV_string = b"A,B\n1.0,1\na,1"

In [ ]:
pl.read_csv(mixed_CSV_string)

In this case Polars casts the column to string dtype.

### Ignore errors
We can also tell Polars to ignore errors in which case values that cannot be cast to the schema for that column are returned as `null`.

In this case we try to make column `A` boolean but have `True` in the first row and `0` in the second row

In [ ]:
CSV_string = b"A,B\nTrue,1\n0,1\n"

(
    pl.read_csv(
        CSV_string,
        schema_overrides={'A':pl.Boolean},
        ignore_errors=True
    )
)

The `DataFrame` has a `null` where the `0` in the second row was.

## Set values to `null`
We might know that there are values in a column that are incorrect.

In this case the CSV has `NaN` in column `B.` We set this to `null` with `null_values`

In [ ]:
CSV_string = b"A,B\nTrue,1\nFalse,NaN\n"

(
    pl.read_csv(
        CSV_string,
        null_values="NaN"
    )
)

We can also pass a list of strings to `null_values` or specify different values to set as `null` for different columns with a `dict`

In [ ]:
CSV_string = b"A,B\nTrue,1\nFalse,NaN\n"

(
    pl.read_csv(
        CSV_string,
        null_values={"B":"NaN"}
    )
)

## Performance of CSV parsing
### Number of threads
The CSV parser in Polars is multithreaded and uses the same number of threads as there are cores on your computer.

We can vary the number of threads with the `n_threads` argument. We can use fewer threads to reduce CPU usage or more threads to (potentially) reduce read time.

In experiments on my computer (8 cores) with different datasets compared to the default:
- reducing `n_threads` to 1 increases time taken by 3x
- reducing `n_threads` to 3-4 increases time taken by 30%
- increasing `n_threads` to 40 reduces time taken by 30% on some datasets but makes no difference on others

In general setting `n_threads` is mainly useful if you want to slow down CSV parsing to avoid impacting other processes on the machine.

### Memory usage
We can potentially reduce memory usage when reading a large CSV with `low_memory = True`. When reading a large CSV Polars reads the CSV into separate chunks in memory before combining the chunks into a `DataFrame` that is a single chunk in memory.

With `low_memory = True` Polars uses a slower non-parallel method of combining the chunks into a `DataFrame` that is a single chunk in memory.

We can also reduce the size of each batch read by the parallel CSV reader with `batch_size`. This may reduce memory pressure where each line is very long or has variable length (e.g. when there is text on each line with variable length)

In [ ]:
(
    pl.read_csv(
        csv_file,
        low_memory=True,
        batch_size=1e5
    )
)               

Be aware that these settings only make a marginal difference to the amount of memory used. If you are trying to work with a CSV that is just too big for your machine then it is best to use streaming mode as explored later in this Section.

## Exercises
In the exercises you will develop your understanding of:
- setting the column names of a CSV
- parsing a CSV
- setting the dtypes
- modifying the number of threads

### Exercise 1
In this exercise we want to parse the CSV strings to produce a `DataFrame` equal to the following

In [ ]:
target = pl.DataFrame({"a":[1,2],"b":[3,4],"c":[5,6]})
target

Parse the CSV strings in the following cells

In [ ]:
CSV_string = b"Data passed quality control 2020-01-01\na,b,c\n1,3,5\n2,4,6\n"
pl.read_csv(<blank>)

In [ ]:
# Rename columns
CSV_string = b"A,B,C\n1,3,5\n2,4,6\n"
pl.read_csv(<blank>)

In [ ]:
# Whitespace delimiter
CSV_string = b"a b c\n1 3 5\n2 4 6\n"
pl.read_csv(<blank>)

In [ ]:
# Comment line
CSV_string = b"a,b,c\n#Data passed quality control 2020-01-01\n1,3,5\n2,4,6\n"
pl.read_csv(<blank>)

This time parse the CSV to produce a `DataFrame` with all columns as 64-bit floats

In [ ]:
CSV_string = b"a,b,c\n#Data passed quality control 2020-01-01\n1,3,5\n2,4,6\n"
pl.read_csv(<blank>)

Find missing data in the CSV and replace with `null`. Ensure columns are not cast to string

In [ ]:
CSV_string = b"a,b,c\n1,3,5\nNA,4,na\n"
pl.read_csv(<blank>)

## Exercise 2
Parse the NYC taxi CSV with:
- the default number of threads,
- one thread and
- 40 threads
to see if it affects performance.

(This dataset is too small to see any differences - but try it with your own datasets to see if changing the number of threads affects performance)

In [ ]:
nyccsv_file = "../data/nyc_trip_data_1k.csv"

In [ ]:
%%timeit -n1 -r3
pl.read_csv(<blank>)

## Solutions

### Solution to exercise 1
In this exercise we want to parse the CSV strings to produce a `DataFrame` equal to the following

In [ ]:
target = pl.DataFrame({"a":[1,2],"b":[3,4],"c":[5,6]})
target

Parse the CSV strings in the following cells

In [ ]:
CSV_string = b"Data passed quality control 2020-01-01\na,b,c\n1,3,5\n2,4,6\n"
pl.read_csv(CSV_string,skip_rows=1)

In [ ]:
CSV_string = b"A,B,C\n1,3,5\n2,4,6\n"
pl.read_csv(CSV_string,new_columns=["a","b","c"])

In [ ]:
CSV_string = b"a b c\n1 3 5\n2 4 6\n"
pl.read_csv(CSV_string,separator=" ")

In [ ]:
CSV_string = b"a,b,c\n#Data passed quality control 2020-01-01\n1,3,5\n2,4,6\n"
pl.read_csv(CSV_string,comment_prefix="#")

This time parse the CSV with all columns as 64-bit floats

In [ ]:
CSV_string = b"a,b,c\n#Data passed quality control 2020-01-01\n1,3,5\n2,4,6\n"
pl.read_csv(CSV_string,comment_prefix="#",schema_overrides={"a":pl.Float64,"b":pl.Float64,"c":pl.Float64})

Find missing data and replace with `null`. Ensure columns are not cast to string

In [ ]:
CSV_string = b"a,b,c\n1,3,5\nNA,4,na\n"
pl.read_csv(CSV_string,null_values={"a":"NA","c":"na"})

## Solution to exercise 2
Parse the NYC taxi CSV with:
- the default number of threads
- one thread
- 40 threads
to see if it affects performance.

Then try it with your own datasets to see if it affects performance

In [ ]:
nyccsv_file = "../data/nyc_trip_data_1k.csv"

In [ ]:
%%timeit -n1
pl.read_csv(nyccsv_file,n_threads=1)

In [ ]:
%%timeit -n1
pl.read_csv(nyccsv_file)

In [ ]:
%%timeit -n1
pl.read_csv(nyccsv_file,n_threads=40)